This is a testing notebook for plotting and fitting FRAP data, using the output of the TrackMate plugin to plot the intensity of a diffusing spot and the output of my "save_background.py" ImageJ script for ROI-based background correction.

In [ ]:
# load the custom modules

%load_ext autoreload # uncomment for debugging
%autoreload 2 # uncomment for debugging

import os, sys, inspect
import matplotlib.pylab as plt
import numpy as np
from pprint import pprint

# Add source code directory (src) to path to enable module import
curr_frame = inspect.getfile(inspect.currentframe())
curr_dir = os.path.dirname(os.path.abspath(curr_frame))
parent_dir = os.path.dirname(curr_dir)
module_dir = os.path.join(parent_dir, 'src')
os.sys.path.insert(0, module_dir)

import parse_trackmate as pt
import fraptools as frap


In [ ]:
# Load test data

# Path to raw TrackMate data
data_dir = '../data/raw/test_data_2_FRAP'
file_name = '20170502-02_cell_00_spot01.xml'
full_file = os.path.join(data_dir, file_name)

# Path to background correction json file
bkgnd_file_name = '20170502-02_cell_00.json'
bkgnd_full_file = os.path.join(data_dir, bkgnd_file_name)

# Read data from the provided source files
raw_data = pt.read_trackmate_file(full_file)
bkgnd_data = frap.read_ij_intensity(bkgnd_full_file)


In [ ]:
# Parse the data into tracks, creating lists of spots that
# make up each individual track 

raw_tracks = pt.get_raw_tracks(raw_data)
spots = pt.list_spots(raw_data)
processed_tracks = pt.build_spot_lists_by_track(raw_tracks, spots)
track_coords, track_edges, track_spot_ids = processed_tracks

# Get intensity values for each frame of each track
all_track_intensities = pt.pull_property_by_track('MEAN_INTENSITY',
                                               track_spot_ids, spots)

# Check that spot radii are constant (otherwise comparison of 
# mean intensities is meaningless)
all_spot_radii = pt.pull_property_by_track('RADIUS',
                                            track_spot_ids, spots)
radii_norm = np.ravel(all_spot_radii - all_spot_radii[0])
diff_radii = np.count_nonzero(radii_norm)
if diff_radii > 0: print('WARNING: spot sizes are not uniform')

(bkgnd_int, bkgnd_corr_int) = frap.correct_background (bkgnd_data, 
                                track_coords, all_track_intensities)


In [ ]:
# Fit corrected intensities to FRAP recovery curves and plot results
%matplotlib

# Input frame interval
frame_interval = 0.2 # in seconds
bleach_n_frames = 2 # how many frames bleaching takes

# Fit data
(fit, data) = frap.fit_frap(bkgnd_corr_int, frame_interval, bleach_n_frames)

# Plot results
for f,d in zip(fit, data):
    frap.plot_fit_results(f,d)
    plt.show()

In [ ]:
# Plot results

#%matplotlib inline
import matplotlib.pylab as plt
#plt.xkcd()

# Various plotting tools
f2, axarr = plt.subplots(1,3, sharey=True)
axarr[0].set_title('Raw intensity values')
axarr[0].set_xlabel('Frame')
axarr[0].set_ylabel('Intensity (A.U.)')
axarr[1].set_xlabel('Frame')
axarr[1].set_ylabel('Corrected intensity (A.U.)')
axarr[1].set_title('Corrected intensity values')
axarr[2].set_xlabel('Frame')
axarr[2].set_ylabel('Background intensity (A.U.)')
axarr[2].set_title('Background intensity values')
for i, track in enumerate(track_coords):
    axarr[0].plot(track[:,2], all_track_intensities[i])
    axarr[1].plot(track[:,2], bkgnd_corr_int[i])
    axarr[2].plot(track[:,2], bkgnd_int[i])
plt.show()
#plt.plot(track_coords[testind][:,2], all_track_intensities[testind])